In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

2022-10-19 12:54:43.715399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-19 12:54:55.103565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-19 12:54:55.103602: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-19 12:54:55.976013: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-19 12:55:13.018350: W tensorflow/stream_executor/platform/de

In [2]:
df = pd.read_csv('assets/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance, test_size=0.2, random_state=25)

In [4]:
X_train

,age,affordibility
0,22,1
13,29,0
6,55,0
17,58,1
24,50,1
19,18,1
25,54,1
16,25,0
20,21,1
3,52,0


In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [6]:
X_train_scaled.head()

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1


In [7]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train_scaled,y_train,epochs=50)
# needs to be done 5000 epochs

2022-10-19 12:55:40.397892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-19 12:55:40.422168: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-19 12:55:40.422317: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (karthikeya): /proc/driver/nvidia/version does not exist
2022-10-19 12:55:40.499958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 12ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 6ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/50
1/1 [==============================] - 0s 7ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/50
1/1 [======================

In [8]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 258ms/step - loss: 0.7108 - accuracy: 0.5000


[0.7107908129692078, 0.5]

In [9]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 128ms/step


array([[0.7938353 ],
       [0.74505764],
       [0.54927635],
       [0.7626963 ],
       [0.79693055],
       [0.814774  ]], dtype=float32)

In [11]:
coef, intercept = model.get_weights()

In [12]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))

In [17]:
def prediction_function(age, affordibility):
    w1, w2 = coef
    weighted_sum = w1*age + w2*affordibility + intercept
    return sigmoid(weighted_sum)

In [14]:
prediction_function(0.6,0)

0.6273982274706037

In [15]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))

In [16]:
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [18]:
epsilon = 1e-15
def log_loss(y_true, y_pred):
    y_pred_new = [max(i,epsilon) for i in y_pred]
    y_pred_new = [min(i,1-epsilon)  for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    BCE = -np.mean(y_true*np.log(y_pred_new)+(1-y_true)*np.log(1-y_pred_new))
    return BCE

In [25]:
def gradient_descent(age,affordibility, y_true, epochs, loss_threshold):
    # w1, w2, bias
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_pred = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_pred)

        d_w1 = (1/n)*np.dot(np.transpose(age),(y_pred-y_true))
        d_w2 = (1/n)*np.dot(np.transpose(affordibility),(y_pred-y_true))
        d_bias = np.mean(y_pred-y_true)

        w1 = w1 - rate * d_w1
        w2 = w2 - rate * d_w2
        bias = bias - rate * d_bias

        print(f'epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss <= loss_threshold:
            break

    return w1, w2, bias

In [26]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000, 0.4631)

epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.681264778737757
epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:0.6591474252715025
epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:0.6431523291301917
epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:0.6316873063379158
epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:0.623471707997592
epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:0.6175321183044205
epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:0.6131591858705934
epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:0.6098518179750948
epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bia

(5.051047623653049, 1.4569794548473887, -2.9596534546250037)